### Downloads the data set

In [8]:
%%time 
import numpy as np                                          # load modules
import pandas as pd
import yfinance as yf

start_date, end_date = "2014-01-01", "2022-06-30"      # specifies time length
                                                    # Entries here should be entered as desired                              
number_of_sp = 1                                # desired number of simultaneous parents                                    
K = N = 2000                           # size of simulation samples
tickers = ['BTI.JO', 'GLN.JO', 'CFR.JO', 'AGL.JO', 'NPN.JO', 'AMS.JO', 'FSR.JO', 'SBK.JO', 'MTN.JO', 'VOD.JO',
           'SOL.JO', 'CPI.JO', 'KIO.JO', 'IMP.JO', 'ABG.JO', 'BAW.JO', 'SHP.JO', 'NED.JO', 'ANG.JO', 'DSY.JO',
           'BVT.JO', 'CLS.JO', 'EXX.JO', 'REM.JO', 'APN.JO', 'PSG.JO', 'WHL.JO', 'NRP.JO', 'MRP.JO', 'ARI.JO',
           'TKG.JO', 'SAP.JO', 'SNT.JO', 'PIK.JO', 'TCP.JO', 'SPP.JO', 'INL.JO', 'TBS.JO', 'TRU.JO', 'AVI.JO']

all_stocks = tickers                       # assign all_stocks the full list of stocks, to be used later

Close = pd.DataFrame(columns = tickers)                           # ensures that data comes in a data frame

for ticker in tickers:                                           # downloads prices of stocks from Yahoo! Finance
    Close[ticker] = yf.download(ticker, start_date, end_date, progress = False)['Close']

Close[2151:2152] = Close[2151:2152].multiply(np.nan) # changes entire row to nan
   
print('Missing stock prices before filling gaps =', Close.isnull().sum().sum())
Close = Close.interpolate(limit_direction = 'both')                  # fills missing values using linear interpolation
print('Missing stock prices after filling gaps =', Close.isnull().sum().sum())  
print('Lenght of closing prices = ', len(Close))
print(" ")

returns = np.log(Close[tickers]).diff()      # calculates log-returns
data = returns[1:]
print("TABLE SHOWING  CLOSING PRICES")
print(Close[0:5])
print(" ")
print("TABLE SHOWING  LOG-RETURNS")
print(data[0:5])

Missing stock prices before filling gaps = 42
Missing stock prices after filling gaps = 0
Lenght of closing prices =  2162
 
TABLE SHOWING  CLOSING PRICES
             BTI.JO  GLN.JO   CFR.JO   AGL.JO        NPN.JO   AMS.JO  FSR.JO  \
Date                                                                           
2014-01-01  56013.0  5481.0  10458.0  22900.0  68860.992188  39391.0  3589.0   
2014-01-02  56246.0  5409.0  10700.0  22699.0  69487.398438  39701.0  3571.0   
2014-01-03  55909.0  5349.0  10499.0  22500.0  68153.546875  39900.0  3550.0   
2014-01-06  56144.0  5364.0  10490.0  22406.0  68002.757812  39372.0  3462.0   
2014-01-07  55757.0  5370.0  10245.0  22239.0  67855.109375  38765.0  3500.0   

             SBK.JO   MTN.JO   VOD.JO  ...  TKG.JO  SAP.JO   SNT.JO  PIK.JO  \
Date                                   ...                                    
2014-01-01  12942.0  21702.0  13300.0  ...  2800.0  3275.0  18628.0  5200.0   
2014-01-02  13031.0  21714.0  13261.0  ...  283

### Selects the simultaneous parent(s)

In [9]:
%%time
from scipy.linalg import block_diag

beta, delta_phi, delta_gamma, time_length_1  = 0.922, 0.993, 0.953, range(0, 782)             # discount factors 

theta_list = []

for ticker in tickers:
    index = tickers.index(ticker)
    
    a = np.zeros((len(tickers), 1))                             # initialise the prior
    R_list, diag_element = [0.0001], 0.01
    R_list.extend([diag_element]*(len(tickers)-1))
    R = np.diag(R_list)
    c, r = 0.001, 5

    for t in time_length_1:
    
        F_list = [1]                   # formation of F. All the remaining stocks are sp of a particular stock   
        data_points = data.iloc[[t]].values.tolist()[0] 
        data_points.pop(index)                         
        F_list.extend(data_points)  
        F = np.array([[i] for i in F_list])  
 
        f = F.transpose() @ a
        q = (F.transpose() @ R) @ F + c            # Kalman filter equatioins for forecasting and updating 
        e = data[ticker][t] - f
        A = (R @ F)/q        
        z = (r + (e**2)/q)/(r+1)
        m = a +A*e
        C = (R-(A @ A.transpose())*q)*z
        n = r+1 
        s = z*c 
        
        a =  m                                           # evolution equations
        block_1 = [[C[0][0] * (1/delta_phi-1)]]          # block discounting 
        block_2 = C[1:, 1:] * (1/delta_gamma-1)
        W  = block_diag(block_1, block_2)
        R = C + W
        c = s 
        r = beta*n
        
    m = m.flatten()                          # convert the posterior mean column vector into a row vector 
    theta_list.append(np.absolute(m))
theta_list                          # list of posteriors for all stocks on the last day of time_length_1

CPU times: total: 7.78 s
Wall time: 7.9 s


[array([0.00057566, 0.0375154 , 0.12906517, 0.0876151 , 0.01653977,
        0.08698779, 0.17285697, 0.00865135, 0.0381114 , 0.20588917,
        0.03424769, 0.11674155, 0.05122547, 0.02389   , 0.13553336,
        0.00982168, 0.12292374, 0.11902198, 0.0217863 , 0.0461972 ,
        0.19438513, 0.14189204, 0.02179454, 0.23753149, 0.08314763,
        0.09814811, 0.11665864, 0.06024798, 0.03850263, 0.00388535,
        0.03315503, 0.06069764, 0.04505588, 0.17913755, 0.21162268,
        0.17248291, 0.42958669, 0.20352936, 0.01429111, 0.38926236]),
 array([6.33965851e-04, 7.59951697e-02, 1.72243448e-01, 6.81483409e-01,
        4.36284891e-02, 3.97413704e-02, 1.05173329e-01, 1.27984632e-01,
        9.98101247e-02, 1.82220688e-01, 2.25928846e-02, 3.67624501e-01,
        5.31812018e-02, 3.98572720e-02, 2.73694278e-01, 7.34345468e-02,
        2.89951415e-01, 4.41316397e-01, 9.26554813e-02, 1.50641126e-01,
        2.00305172e-01, 5.94740007e-01, 1.26082876e-01, 5.59322068e-01,
        1.82392369e-01

### Obtains beta 

In [13]:
%%time 
from scipy import special 
from scipy.optimize import fsolve
from scipy.stats import multivariate_t
from math import gamma, pi

df_range = np.linspace(0.825, 0.985, num = 5)   # interval over which beta is chosen 


LLH_all_t_df_list = []
for beta in df_range:
    
    delta_phi, delta_gamma, time_length_2, M = 0.993, 0.953, range(782, 1288), len(tickers)
    
    r_list, c_list, a, R_values, diag_element =  [5], [0.001], np.zeros((number_of_sp+1), dtype = int ), [0.0001], 0.01
    
    r_list.extend(r_list*(M-1))          # Initial prior: a, R, r, c
    c_list.extend(c_list*(M-1))
    
    a = np.array([[i] for i in a])
    a_list = [a]
    a_list.extend(a_list*(M-1))
    
    R_values.extend([diag_element]*number_of_sp)
    R = np.diag(R_values)
    R_list = [R]
    R_list.extend(R_list*(M-1))
    
    LLH_all_t_list = []
    for t in time_length_2:

        n_list = []
        all_sp_sets_IS = []
        all_lamda_IS_lists = []
        all_theta_IS_lists = []
        r_LLH_list = []
        f_LLH_list = []
        q_LLH_list = []
    
        for ticker in tickers:
            index = tickers.index(ticker) 
            
            a, R, r, c = a_list[index], R_list[index], r_list[index], c_list[index]    

            r_LLH_list.append(r) # keep a list of r values to be used to calculate loglikelihood
            
            gamma_values = theta_list[index][1:]      
            data_sp = data.drop([ticker], axis = 1)         
            tickers.remove(ticker)                               
            F_list = [data_sp[tickers[gamma_values.argsort()[-number_of_sp:][k]]][t] 
                      for k in range((number_of_sp-1), -1, -1)] #generate F vector by adding values of simultaneous parents
            F_list = [1] + F_list
            F = np.array([ [i] for i in F_list])
            all_stocks.insert(index, ticker)                 
    
            f = F.transpose() @ a        # Kalman filter equations for forecasting and  updating to naive posterior
            f_LLH_list.append(f[0][0])
            q = F.transpose() @ R @ F + c    
            q_LLH_list.append(q[0][0]) 
            e = data[ticker][t] - f 
            A = (R @ F)/q
            z = (r + (e**2)/q)/(r+1)
            m = a +A*e
            C = (R-(A @ A.transpose())*q)*z
            n = r + 1
            n_list.append(n)
            s = z*c; s = s[0][0]
        
            lamda_IS_list = [np.random.gamma( (n/2),  (2/(n*s)), N)][0]        # simulation of precision lambda for IS 
            all_lamda_IS_lists.append(lamda_IS_list)
                                                   # IMPORTANCE SAMPLING. Generate an IS for all states and precisions 
            single_ticker_sp_sets_IS = []
            theta_IS_list = []
            for i in range(0, N):
                
                # simulation of state vector theta given precision lambda                                                                                                                                                                            
                theta_IS = np.random.multivariate_normal(m.flatten(), C/(s*lamda_IS_list[i]), check_valid = 'warn')
                theta_IS_list.append(theta_IS.reshape(-1, 1))
                
                sp_set_IS = [data[ticker][t] for ticker in tickers]    # returns for all stocks at time t
                theta_IS = theta_IS_list[i]
                
                for i in range(0, M):
                    for k in range(1, (number_of_sp+1)):                   # formation of a row of the gamma matrix 
                        if sp_set_IS[i] == F[k][0]:
                            sp_set_IS[i] = theta_IS[k][0]               
                for i in range(0, M):
                    u = 0
                    for k in range(1, (number_of_sp+1)): 
                        if sp_set_IS[i] == theta_IS[k][0]:
                            u = u + 1
                    if u == 0:
                        sp_set_IS[i] = 0
                single_ticker_sp_sets_IS.append(sp_set_IS)            
            all_sp_sets_IS.append(single_ticker_sp_sets_IS)
            all_theta_IS_lists.append(theta_IS_list)
            
        IS_weight_list = []
        for T in range(0, N):                                              # formation of the Gamma_t matrix in the IS     
            Gamma_rows = [all_sp_sets_IS[j][T] for j in range(0, M)]    
            Gamma_IS = np.array(Gamma_rows)                                    # Gamma_IS is the gamma matrix in the IS  
            IS_weight = np.absolute(np.linalg.det( np.identity(M) - Gamma_IS ))                   # IS weights 
            IS_weight_list.append(IS_weight)
        IS_weight_array = np.array(IS_weight_list)
        alpha = (1/np.sum(IS_weight_array))*IS_weight_array                         # normalised IS weights (alpha)
        
        LLH_t_list = []
        for ticker in tickers: 
            index = tickers.index(ticker)              
            e = data[ticker][t] - f_LLH_list[index]   # get forecast error, degrees of freedom and scale for each stock 
            deg_fre = r_LLH_list[index]
            scale_value = q_LLH_list[index]
            
            # predictive distribution density used to set up loglikelihood 
            part_1 = np.log( gamma((deg_fre+1)/2) / (gamma(deg_fre/2)* (deg_fre*pi*scale_value)**0.5 ) )
            part_2 =  ((deg_fre + 1) /2) * np.log( 1 + (e**2)/(deg_fre*scale_value))
            LLH_t  = part_1 - part_2        # loglikelihood at time t for each stock 
            LLH_t_list.append(LLH_t)
        
        LLH_all_t_list.append(LLH_t_list)
            
        n_list_2 = []
        a_temp_list = []
        R_temp_list = []
        r_temp_list = []
        c_temp_list = []
                              # VB starts here 
        for ticker in tickers: 
            index = tickers.index(ticker)                                          
            lamda_IS_list  = all_lamda_IS_lists[index] # the list of all lambda values in the IS for a single stock 
            theta_IS_list  = all_theta_IS_lists[index] # list of all theta values in the IS for a single stock

            m_num_list = [alpha[i]*lamda_IS_list[i]*theta_IS_list[i] for i in range(0, N)]
            m_den_list = [alpha[i]*lamda_IS_list[i] for i in range(0, N)]
            m = sum(m_num_list)/sum(m_den_list)              # Calculation of m from the VB formula 
            
                         
            V = sum([alpha[i] * lamda_IS_list[i] * (theta_IS_list[i]-m) @ (theta_IS_list[i]-m).transpose()
                     for i in range(0, N) ])   # calculation of V using the VB formula 
           
            d = sum([alpha[i]*lamda_IS_list[i]*((theta_IS_list[i] - m).transpose() @ np.linalg.pinv(V) @ 
                    (theta_IS_list[i] - m)) for i in range(0, N)])[0][0]     # calculation of d
            
            p_j = len(theta_IS)           # size of state vector in phases 2 and 3 
    
            n_initial = n_list[index]          # calculation of n 
            sum_1 = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N) ])
            sum_2 = sum([alpha[i] * np.log(lamda_IS_list[i]) for i in range(0, N) ])
            func = lambda n: (np.log(abs(n+p_j - d)) - special.polygamma(0, (n/2)) - (p_j - d)/n -
                              np.log(abs( 2 * sum_1)) + sum_2)
            n =  fsolve(func, [n_initial])[0]
            n_list_2.append(n)
            
            sum_in_s = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N)]) # calculation of s
            s =(n + p_j - d)/(n*sum_in_s) 
    
            C = s*V                     # Calculation of C
            
            a =  m                          # kalman filter equations, evolution to the next day 
            a_temp_list.append(a)           # parameters are stored in temporary lists named as temp
            
            block_1 = [[C[0][0] * (1/delta_phi-1)]]          # block discounting 
            block_2 = C[1:, 1:] * (1/delta_gamma-1)
            W  = block_diag(block_1, block_2)

            R = C + W
            R_temp_list.append(R) 
            r = beta*n
            r_temp_list.append(r)
            c = s
            c_temp_list.append(c)
                      
        a_list = a_temp_list       # fetch parameters from temporaly lists to move to the next value of t 
        R_list = R_temp_list
        r_list = r_temp_list
        c_list = c_temp_list
    LLH_all_t_df_list.append(LLH_all_t_list)

all_LLH = []                       # LLH_all_t_df_list is a list of lists of LLH values of all stocks for
for i in LLH_all_t_df_list:          # different discount factors at different times 
    LLH_each_df = []                
    for ticker in tickers:             # We want to sum up (independently) LLH values for each stock for the
        index = tickers.index(ticker)    #  entire period for every discount factor
        LLH_list = [k[index] for k in i]
        LLH = sum(LLH_list)
        LLH_each_df.append(LLH)
    all_LLH.append(LLH_each_df)  # all_LLH is the list with loglikelihood for each stock at different discount factors
df_value_list = []
for ticker in tickers: 
    index = tickers.index(ticker)
    single_stock_LLH_list = [ k[index] for k in all_LLH ]
    df_value = df_range[np.argmax(single_stock_LLH_list)]
    df_value_list.append(df_value)
print('beta values =', df_value_list)
ave_beta =  sum(df_value_list)/M
print('Average value of beta is ', round(ave_beta, 5))

beta values = [0.865, 0.945, 0.945, 0.985, 0.945, 0.905, 0.905, 0.985, 0.865, 0.865, 0.945, 0.905, 0.985, 0.945, 0.945, 0.945, 0.905, 0.945, 0.985, 0.945, 0.945, 0.945, 0.905, 0.945, 0.825, 0.865, 0.905, 0.825, 0.905, 0.945, 0.905, 0.985, 0.945, 0.945, 0.905, 0.905, 0.905, 0.905, 0.985, 0.905]
Average value of beta is  0.924
CPU times: total: 19h 54min 29s
Wall time: 19h 56min 42s


### Obtains delta_phi

In [11]:
%%time 
import math
from scipy import special 
from scipy.optimize import fsolve
from scipy.stats import multivariate_t
from math import gamma
from math import pi

df_range = np.linspace(0.959, 0.999, num = 5)                       # interval over which delta_phi is chosen 


LLH_all_t_df_list = []
for delta_phi in df_range:
    
    beta, delta_gamma, time_length_2, M = 0.922, 0.953, range(782, 1288), len(tickers)
    
    r_list, c_list, a, R_values, diag_element =  [5], [0.001], np.zeros((number_of_sp+1), dtype = int ), [0.0001], 0.01
    
    r_list.extend(r_list*(M-1))          # Initial prior: a, R, r, c
    c_list.extend(c_list*(M-1))
    
    a = np.array([[i] for i in a])
    a_list = [a]
    a_list.extend(a_list*(M-1))
    
    R_values.extend([diag_element]*number_of_sp)
    R = np.diag(R_values)
    R_list = [R]
    R_list.extend(R_list*(M-1))
    
    LLH_all_t_list = []
    for t in time_length_2:

        n_list = []
        all_sp_sets_IS = []
        all_lamda_IS_lists = []
        all_theta_IS_lists = []
        r_LLH_list = []
        f_LLH_list = []
        q_LLH_list = []
    
        for ticker in tickers:
            index = tickers.index(ticker)      
            
            a, R, r, c = a_list[index], R_list[index], r_list[index], c_list[index] # priors for individual stocks   

            r_LLH_list.append(r) # keep a list of r values to be used to calculate loglikelihood
            
            gamma_values = theta_list[index][1:]      # obtains gamma values to guide in obtaining simultaneous parents
            data_sp = data.drop([ticker], axis = 1)         # drop the value of the current stock in the dataframe 
            tickers.remove(ticker)                          # remove its symbol from the columns       
            F_list = [data_sp[tickers[gamma_values.argsort()[-number_of_sp:][k]]][t] 
                      for k in range((number_of_sp-1), -1, -1)] #generate F vector by adding values of simultaneous parents
            F_list = [1] + F_list
            F = np.array([ [i] for i in F_list])
            all_stocks.insert(index, ticker)                    # get back full list of stocks   
    
            f = F.transpose() @ a        # Kalman filter equations for forecasting and  updating to naive posterior
            f_LLH_list.append(f[0][0])
            q = F.transpose() @ R @ F + c    
            q_LLH_list.append(q[0][0]) 
            e = data[ticker][t] - f 
            A = (R @ F)/q
            z = (r + (e**2)/q)/(r+1)
            m = a +A*e
            C = (R-(A @ A.transpose())*q)*z
            n = r + 1
            n_list.append(n)
            s = z*c; s = s[0][0]
        
            lamda_IS_list = [np.random.gamma( (n/2),  (2/(n*s)), N)][0]        # simulation of precision lambda for IS 
            all_lamda_IS_lists.append(lamda_IS_list)                
                                                   # IMPORTANCE SAMPLING. Generate an IS for all states and precisions 
            single_ticker_sp_sets_IS = [] 
            theta_IS_list = []
            for i in range(0, N):
                
                # simulation of state vector theta given precision lambda                                                                                                                                                                            
                theta_IS = np.random.multivariate_normal(m.flatten(), C/(s*lamda_IS_list[i]), check_valid = 'warn')
                theta_IS_list.append(theta_IS.reshape(-1, 1))
                
                sp_set_IS = [data[ticker][t] for ticker in tickers]    # returns for all stocks at time t
                theta_IS = theta_IS_list[i]
                
                for i in range(0, M):
                    for k in range(1, (number_of_sp+1)):                   # formation of a row of the gamma matrix 
                        if sp_set_IS[i] == F[k][0]:
                            sp_set_IS[i] = theta_IS[k][0]               
                for i in range(0, M):
                    u = 0
                    for k in range(1, (number_of_sp+1)): 
                        if sp_set_IS[i] == theta_IS[k][0]:
                            u = u + 1
                    if u == 0:
                        sp_set_IS[i] = 0
                single_ticker_sp_sets_IS.append(sp_set_IS)
                            
            all_sp_sets_IS.append(single_ticker_sp_sets_IS)
            all_theta_IS_lists.append(theta_IS_list)
            
        IS_weight_list = []
        for T in range(0, N):                                              # formation of the Gamma_t matrix in the IS     
            Gamma_rows = [all_sp_sets_IS[j][T] for j in range(0, M)]    
            Gamma_IS = np.array(Gamma_rows)                                    # Gamma_IS is the gamma matrix in the IS  
            IS_weight = np.absolute(np.linalg.det( np.identity(M) - Gamma_IS ))                   # IS weights 
            IS_weight_list.append(IS_weight)
        IS_weight_array = np.array(IS_weight_list)
        alpha = (1/np.sum(IS_weight_array))*IS_weight_array                         # normalised IS weights (alpha)
        
        LLH_t_list = []
        for ticker in tickers: 
            index = tickers.index(ticker)              
            e = data[ticker][t] - f_LLH_list[index]   # get forecast error, degrees of freedom and scale for each stock 
            deg_fre = r_LLH_list[index]
            scale_value = q_LLH_list[index]
            
            # predictive distribution density used to set up loglikelihood 
            part_1 = np.log( gamma((deg_fre+1)/2) / (gamma(deg_fre/2)* (deg_fre*pi*scale_value)**0.5 ) )
            part_2 =  ((deg_fre + 1) /2) * np.log( 1 + (e**2)/(deg_fre*scale_value))
            LLH_t  = part_1 - part_2        # loglikelihood at time t for each stock 
            LLH_t_list.append(LLH_t)
        
        LLH_all_t_list.append(LLH_t_list)
            
        n_list_2 = []
        a_temp_list = []
        R_temp_list = []
        r_temp_list = []
        c_temp_list = []
                              # VB starts here 
        for ticker in tickers: 
            index = tickers.index(ticker)  # assigns positions/numbers to stocks in the list of stocks                                        
            lamda_IS_list  = all_lamda_IS_lists[index] # the list of all lambda values in the IS for a single stock 
            theta_IS_list  = all_theta_IS_lists[index] # list of all theta values in the IS for a single stock

            m_num_list = [alpha[i]*lamda_IS_list[i]*theta_IS_list[i] for i in range(0, N)]
            m_den_list = [alpha[i]*lamda_IS_list[i] for i in range(0, N)]
            m = sum(m_num_list)/sum(m_den_list)              # Calculation of m from the VB formula 
            
                         
            V = sum([alpha[i] * lamda_IS_list[i] * (theta_IS_list[i]-m) @ (theta_IS_list[i]-m).transpose()
                     for i in range(0, N) ])   # calculation of V using the VB formula 
           
            d = sum([alpha[i]*lamda_IS_list[i]*((theta_IS_list[i] - m).transpose() @ np.linalg.pinv(V) @ 
                    (theta_IS_list[i] - m)) for i in range(0, N)])[0][0]     # calculation of d
            
            p_j = len(theta_IS)           # size of state vector in phases 2 and 3 
    
            n_initial = n_list[index]          # calculation of n 
            sum_1 = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N) ])
            sum_2 = sum([alpha[i] * np.log(lamda_IS_list[i]) for i in range(0, N) ])
            func = lambda n: (np.log(abs(n+p_j - d)) - special.polygamma(0, (n/2)) - (p_j - d)/n -
                              np.log(abs( 2 * sum_1)) + sum_2)
            n =  fsolve(func, [n_initial])[0]
            n_list_2.append(n)
            
            sum_in_s = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N)]) # calculation of s
            s =(n + p_j - d)/(n*sum_in_s) 
    
            C = s*V                     # Calculation of C
            
            a =  m                          # kalman filter equations, evolution to the next day 
            a_temp_list.append(a)           # parameters are stored in temporary lists named as temp
            
            block_1 = [[C[0][0] * (1/delta_phi-1)]]          # block discounting 
            block_2 = C[1:, 1:] * (1/delta_gamma-1)
            W  = block_diag(block_1, block_2)

            R = C + W
            R_temp_list.append(R) 
            r = beta*n
            r_temp_list.append(r)
            c = s
            c_temp_list.append(c)
                      
        a_list = a_temp_list       # fetch parameters from temporaly lists to move to the next value of t 
        R_list = R_temp_list
        r_list = r_temp_list
        c_list = c_temp_list
    LLH_all_t_df_list.append(LLH_all_t_list)

all_LLH = []                       # LLH_all_t_df_list is a list of lists of LLH values of all stocks for
for i in LLH_all_t_df_list:          # different discount factors at different times 
    LLH_each_df = []                
    for ticker in tickers:             # We want to sum up (independently) LLH values for each stock for the
        index = tickers.index(ticker)    #  entire period for every discount factor
        LLH_list = [k[index] for k in i]
        LLH = sum(LLH_list)
        LLH_each_df.append(LLH)
    all_LLH.append(LLH_each_df)  # all_LLH is the list with loglikelihood for each stock at different discount factors
df_value_list = []
for ticker in tickers: 
    index = tickers.index(ticker)
    single_stock_LLH_list = [k[index] for k in all_LLH ]
    df_value = df_range[np.argmax(single_stock_LLH_list)]
    df_value_list.append(df_value)
print('delta_phi values =', df_value_list)
ave_delta_phi =  sum(df_value_list)/M
print('Average value of delta_phi is ', round(ave_delta_phi, 5))

delta_phi values = [0.999, 0.999, 0.989, 0.999, 0.999, 0.989, 0.989, 0.969, 0.999, 0.999, 0.999, 0.989, 0.999, 0.989, 0.999, 0.999, 0.989, 0.989, 0.979, 0.999, 0.989, 0.999, 0.989, 0.999, 0.989, 0.969, 0.999, 0.999, 0.999, 0.999, 0.999, 0.989, 0.999, 0.999, 0.989, 0.999, 0.999, 0.979, 0.999, 0.999]
Average value of delta_phi is  0.9935
CPU times: total: 20h 10min 40s
Wall time: 20h 16min 58s


### Obtains delta_gamma

In [12]:
%%time 
from scipy import special 
from scipy.optimize import fsolve
from scipy.stats import multivariate_t
from math import gamma, pi

df_range = np.linspace(0.859, 0.999, num = 5)                     # interval over which delta_gamma is chosen 

LLH_all_t_df_list = []
for delta_gamma in df_range:
    
    beta, delta_phi, time_length_2, M =  0.922, 0.993, range(782, 1288), len(tickers)
    
    r_list, c_list, a, R_values, diag_element =  [5], [0.001], np.zeros((number_of_sp+1), dtype = int ), [0.0001], 0.01
    
    r_list.extend(r_list*(M-1))          # Initial prior: a, R, r, c
    c_list.extend(c_list*(M-1))
    
    a = np.array([[i] for i in a])
    a_list = [a]
    a_list.extend(a_list*(M-1))
    
    R_values.extend([diag_element]*number_of_sp)
    R = np.diag(R_values)
    R_list = [R]
    R_list.extend(R_list*(M-1))
    
    LLH_all_t_list = []
    for t in time_length_2:

        n_list = []
        all_sp_sets_IS = []
        all_lamda_IS_lists = []
        all_theta_IS_lists = []
        r_LLH_list = []
        f_LLH_list = []
        q_LLH_list = []
    
        for ticker in tickers:
            index = tickers.index(ticker)  # assigns positions/numbers to stocks in the list of stocks     
            
            a, R, r, c = a_list[index], R_list[index], r_list[index], c_list[index] # priors for individual stocks   

            r_LLH_list.append(r) # keep a list of r values to be used to calculate loglikelihood
            
            gamma_values = theta_list[index][1:]      # obtains gamma values to guide in obtaining simultaneous parents
            data_sp = data.drop([ticker], axis = 1)         # drop the value of the current stock in the dataframe 
            tickers.remove(ticker)                          # remove its symbol from the columns       
            F_list = [data_sp[tickers[gamma_values.argsort()[-number_of_sp:][k]]][t] 
                      for k in range((number_of_sp-1), -1, -1)] #generate F vector by adding values of simultaneous parents
            F_list = [1] + F_list
            F = np.array([ [i] for i in F_list])
            all_stocks.insert(index, ticker)                    # get back full list of stocks   
    
            f = F.transpose() @ a        # Kalman filter equations for forecasting and  updating to naive posterior
            f_LLH_list.append(f[0][0])
            q = F.transpose() @ R @ F + c    
            q_LLH_list.append(q[0][0]) 
            e = data[ticker][t] - f 
            A = (R @ F)/q
            z = (r + (e**2)/q)/(r+1)
            m = a +A*e
            C = (R-(A @ A.transpose())*q)*z
            n = r + 1
            n_list.append(n)
            s = z*c; s = s[0][0]
        
            lamda_IS_list = [np.random.gamma( (n/2),  (2/(n*s)), N)][0]        # simulation of precision lambda for IS 
            all_lamda_IS_lists.append(lamda_IS_list)
                                                   # IMPORTANCE SAMPLING. Generate an IS for all states and precisions 
            single_ticker_sp_sets_IS = []
            theta_IS_list = []
            for i in range(0, N):
                
                # simulation of state vector theta given precision lambda                                                                                                                                                                            
                theta_IS = np.random.multivariate_normal(m.flatten(), C/(s*lamda_IS_list[i]), check_valid = 'warn')
                theta_IS_list.append(theta_IS.reshape(-1, 1))
                
                sp_set_IS = [data[ticker][t] for ticker in tickers]    # returns for all stocks at time t
                theta_IS = theta_IS_list[i]
                
                for i in range(0, M):
                    for k in range(1, (number_of_sp+1)):                   # formation of a row of the gamma matrix 
                        if sp_set_IS[i] == F[k][0]:
                            sp_set_IS[i] = theta_IS[k][0]               
                for i in range(0, M):
                    u = 0
                    for k in range(1, (number_of_sp+1)): 
                        if sp_set_IS[i] == theta_IS[k][0]:
                            u = u + 1
                    if u == 0:
                        sp_set_IS[i] = 0
                single_ticker_sp_sets_IS.append(sp_set_IS)            
            all_sp_sets_IS.append(single_ticker_sp_sets_IS)
            all_theta_IS_lists.append(theta_IS_list)
            
        IS_weight_list = []
        for T in range(0, N):                                              # formation of the Gamma_t matrix in the IS     
            Gamma_rows = [all_sp_sets_IS[j][T] for j in range(0, M)]    
            Gamma_IS = np.array(Gamma_rows)                                    # Gamma_IS is the gamma matrix in the IS  
            IS_weight = np.absolute(np.linalg.det( np.identity(M) - Gamma_IS ))                   # IS weights 
            IS_weight_list.append(IS_weight)
        IS_weight_array = np.array(IS_weight_list)
        alpha = (1/np.sum(IS_weight_array))*IS_weight_array                         # normalised IS weights (alpha)
        
        LLH_t_list = []
        for ticker in tickers: 
            index = tickers.index(ticker)              
            e = data[ticker][t] - f_LLH_list[index]   # get forecast error, degrees of freedom and scale for each stock 
            deg_fre = r_LLH_list[index]
            scale_value = q_LLH_list[index]
            
            # predictive distribution density used to set up loglikelihood 
            part_1 = np.log( gamma((deg_fre+1)/2) / (gamma(deg_fre/2)* (deg_fre*pi*scale_value)**0.5 ) )
            part_2 =  ((deg_fre + 1) /2) * np.log( 1 + (e**2)/(deg_fre*scale_value))
            LLH_t  = part_1 - part_2        # loglikelihood at time t for each stock 
            LLH_t_list.append(LLH_t)
        
        LLH_all_t_list.append(LLH_t_list)
            
        n_list_2 = []
        a_temp_list = []
        R_temp_list = []
        r_temp_list = []
        c_temp_list = []
                              # VB starts here 
        for ticker in tickers: 
            index = tickers.index(ticker)  # assigns positions/numbers to stocks in the list of stocks                                        
            lamda_IS_list  = all_lamda_IS_lists[index] # the list of all lambda values in the IS for a single stock 
            theta_IS_list  = all_theta_IS_lists[index] # list of all theta values in the IS for a single stock

            m_num_list = [alpha[i]*lamda_IS_list[i]*theta_IS_list[i] for i in range(0, N)]
            m_den_list = [alpha[i]*lamda_IS_list[i] for i in range(0, N)]
            m = sum(m_num_list)/sum(m_den_list)              # Calculation of m from the VB formula 
            
                         
            V = sum([alpha[i] * lamda_IS_list[i] * (theta_IS_list[i]-m) @ (theta_IS_list[i]-m).transpose()
                     for i in range(0, N) ])   # calculation of V using the VB formula 
           
            d = sum([alpha[i]*lamda_IS_list[i]*((theta_IS_list[i] - m).transpose() @ np.linalg.pinv(V) @ 
                    (theta_IS_list[i] - m)) for i in range(0, N)])[0][0]     # calculation of d
            
            p_j = len(theta_IS)           # size of state vector in phases 2 and 3 
    
            n_initial = n_list[index]          # calculation of n 
            sum_1 = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N) ])
            sum_2 = sum([alpha[i] * np.log(lamda_IS_list[i]) for i in range(0, N) ])
            func = lambda n: (np.log(abs(n+p_j - d)) - special.polygamma(0, (n/2)) - (p_j - d)/n -
                              np.log(abs( 2 * sum_1)) + sum_2)
            n =  fsolve(func, [n_initial])[0]
            n_list_2.append(n)
            
            sum_in_s = sum([alpha[i] * lamda_IS_list[i] for i in range(0, N)]) # calculation of s
            s =(n + p_j - d)/(n*sum_in_s) 
    
            C = s*V                     # Calculation of C
            
            a =  m                          # kalman filter equations, evolution to the next day 
            a_temp_list.append(a)           # parameters are stored in temporary lists named as temp
            
            block_1 = [[C[0][0] * (1/delta_phi-1)]]          # block discounting 
            block_2 = C[1:, 1:] * (1/delta_gamma-1)
            W  = block_diag(block_1, block_2)

            R = C + W
            R_temp_list.append(R) 
            r = beta*n
            r_temp_list.append(r)
            c = s
            c_temp_list.append(c)
                      
        a_list = a_temp_list       # fetch parameters from temporaly lists to move to the next value of t 
        R_list = R_temp_list
        r_list = r_temp_list
        c_list = c_temp_list
    LLH_all_t_df_list.append(LLH_all_t_list)

all_LLH = []                       # LLH_all_t_df_list is a list of lists of LLH values of all stocks for
for i in LLH_all_t_df_list:          # different discount factors at different times 
    LLH_each_df = []                
    for ticker in tickers:             # We want to sum up (independently) LLH values for each stock for the
        index = tickers.index(ticker)    #  entire period for every discount factor
        LLH_list = [k[index] for k in i]
        LLH = sum(LLH_list)
        LLH_each_df.append(LLH)
    all_LLH.append(LLH_each_df)  # all_LLH is the list with loglikelihood for each stock at different discount factors
df_value_list = []
for ticker in tickers: 
    index = tickers.index(ticker)
    single_stock_LLH_list = [ k[index] for k in all_LLH ]
    df_value = df_range[np.argmax(single_stock_LLH_list)]
    df_value_list.append(df_value)
print('delta_gamma values =', df_value_list)
ave_delta_gamma =  sum(df_value_list)/M
print('Average value of delta_gamma is ', round(ave_delta_gamma, 5))

delta_gamma values = [0.999, 0.929, 0.999, 0.999, 0.999, 0.929, 0.929, 0.859, 0.964, 0.964, 0.964, 0.929, 0.999, 0.999, 0.929, 0.999, 0.964, 0.929, 0.999, 0.929, 0.929, 0.929, 0.964, 0.964, 0.964, 0.894, 0.964, 0.964, 0.964, 0.999, 0.999, 0.964, 0.964, 0.964, 0.999, 0.964, 0.964, 0.929, 0.929, 0.964]
Average value of delta_gamma is  0.95962
CPU times: total: 19h 55min 23s
Wall time: 19h 57min 14s
